Para realizar la optimización debemos tener instaladas las librerías que aparecen abajo
en el siguiente segmento de código. Si tienes python instalado con la distribución de anaconda
entonces solo tendrás que installar la librería "Pandas DataReader", para esto puedes ir a la 
terminal y escribir: pip install pandas_datareader.

Una vez tengas las librerias instaladas las importamos como se muestra acontinuación.

In [2]:
import pandas as pd #to manipulaze databframes
import numpy as np # to generate random weigths and linear algebra 
from pandas_datareader import DataReader as wb # to get stocks data from yahoo finance
from scipy.optimize import minimize # to optimize sharpe-ratio
import matplotlib.pyplot as plt # to plot our data
import seaborn as sns # style of our plots

El primer paso es obtener la información desde Yahoo Finance, podríamos descargar uno a uno los datos
de las acciones, luego en excel sincronizar las fechas y cargar el archivo modificado de nuevo a
python. Sin embargo, el proceso de obtener y sincronizar los precios de las acciones se puede hacer
de una forma mucho más facil utilizando la librería "Pandas DataReader", específicamente
la función "DataReader" que cargamos inicialmente commo "wb". Con esto obtendremos los precios de
cada acción en un solo data frame utilizando unas pocas líneas de código.

Para cargar los precios desde Yahoo creamos una variable de tipo lista con los tickers de las acciones de Apple, Facebook, Amazon
y Microsoft, además, creamos dos variables llamadas "start" y "end" donde colocamos las fechas de inicio y
termino respectivamente como strings siguiendo el formato año-mes-día.

Una vez establecidos los inputs necesarios, creamos una variable llamada "df", la igualmos a la  función DataReader
abrebiada como "wb" y le pasamos los imputs. Primero se ponen los tickers, luego escribimos 'yahoo' (en minúscula)
y por pultimo las fechas de inicio y termino, es importante hacerlo en este orden. Una vez cerramos la función le 
agregamos el atributo ['Adj Close'], esto le dice a la función que de toda la información que tiene Yahoo sobre
las acciones, nos retorne solo los precios de cierre ajustados por dividendos, si quisieramos los precios sin ajuste
debemos escrbir el atributo ['Close'].

Esta funcicón nos devuelve un pandas data frame donde las columnas son los tickers de cada acción y el indice son las
fechas de cada día, dentro del data frame tenemos los precios diarios de cada acción.

In [3]:
# Define our stocks in the portfolio
tickers = ['AAPL', 'FB', 'AMZN', 'MSFT']
# Get stock prices in Yahoo Finance
start = '2019-1-1'
end = '2021-1-1'
df = wb(tickers,'yahoo', start, end)['Adj Close']

Si utilizamos la función "print()" y le pasamos el data frame, podremos ver en la consola los datos que obtuvimos de Yahoo

In [4]:
print(df)

Symbols           AAPL          FB         AMZN        MSFT
Date                                                       
2019-01-02   38.382229  135.679993  1539.130005   97.961319
2019-01-03   34.559078  131.740005  1500.280029   94.357529
2019-01-04   36.034370  137.949997  1575.390015   98.746010
2019-01-07   35.954170  138.050003  1629.510010   98.871956
2019-01-08   36.639565  142.529999  1656.579956   99.588829
...                ...         ...          ...         ...
2020-12-24  131.352829  267.399994  3172.689941  221.302490
2020-12-28  136.050766  277.000000  3283.959961  223.498138
2020-12-29  134.239273  276.779999  3322.000000  222.693390
2020-12-30  133.094650  271.869995  3285.850098  220.239456
2020-12-31  132.069473  273.160004  3256.929932  220.974640

[505 rows x 4 columns]


Posteriormenta hay que obetener los retornos diarios de cada acción, en este ejercicio calcularemos
retornos continuos los cuales son el logarítmon natural de la división entre el precio de cada día sobre el precio del
día anterior. Para esto utilizamos la función de numpy "log" que nos permite obtener el logaritmo natural de
lo que coloquemos dentro de esta, en este caso le pasamos el data frame (df) dividido por si mismo pero con la
función "shift(1)" de pandas, con esto estamos dividiendo cada fila del data frame con la fila anterior.

In [5]:
# Calculate continuous returns
df = np.log(df/df.shift(1))

Hacemos un print para ver cómo cambiaron los datos

In [6]:
print(df)

Symbols         AAPL        FB      AMZN      MSFT
Date                                              
2019-01-02       NaN       NaN       NaN       NaN
2019-01-03 -0.104924 -0.029469 -0.025566 -0.037482
2019-01-04  0.041803  0.046061  0.048851  0.045460
2019-01-07 -0.002228  0.000725  0.033776  0.001275
2019-01-08  0.018884  0.031937  0.016476  0.007224
...              ...       ...       ...       ...
2020-12-24  0.007683 -0.002652 -0.003957  0.007797
2020-12-28  0.035141  0.035272  0.034470  0.009873
2020-12-29 -0.013404 -0.000795  0.011517 -0.003607
2020-12-30 -0.008563 -0.017899 -0.010942 -0.011081
2020-12-31 -0.007732  0.004734 -0.008840  0.003333

[505 rows x 4 columns]


Se puede apreciar como ahora en el data frame dejamos de tener precios y ahora tenemos números decimales
entre -1 y 1. Admemás, en la primera fila nos aparecen datos del tipo NaN, esto es porque anteriormente
cañculamos el logaritmo natural de cada precio con el precio del día anterior, obviamente la primera fila
de precios no tenía precios anteriores por lo el resultado para la primera fila nos arroja datos tipo Nan.

Para eliminar esta fila utilizamos la función de pandas "dropna()", la cual nos permite borrar del data frame
las filas que contengan datos del tipo NaN. Además, agregamos inmediatamente un print para ver el data frame

In [7]:
df = df.dropna()
print(df)

Symbols         AAPL        FB      AMZN      MSFT
Date                                              
2019-01-03 -0.104924 -0.029469 -0.025566 -0.037482
2019-01-04  0.041803  0.046061  0.048851  0.045460
2019-01-07 -0.002228  0.000725  0.033776  0.001275
2019-01-08  0.018884  0.031937  0.016476  0.007224
2019-01-09  0.016839  0.011857  0.001713  0.014198
...              ...       ...       ...       ...
2020-12-24  0.007683 -0.002652 -0.003957  0.007797
2020-12-28  0.035141  0.035272  0.034470  0.009873
2020-12-29 -0.013404 -0.000795  0.011517 -0.003607
2020-12-30 -0.008563 -0.017899 -0.010942 -0.011081
2020-12-31 -0.007732  0.004734 -0.008840  0.003333

[504 rows x 4 columns]
